## 1. Basic 3D model: Diffusion limited aggregation 

Our model simulates the growth of a coral structure using the principles of diffusion-limited aggregation. 
- Particles are randomly spawned on a sphere around the cluster.
- These particles perform a random walk until they attach to the existing coral.
- Growth is governed by parameters like stickiness (k_cons) and a minimum radius (rmin) defining the growth region.

This model produces fractal structures as seen in real-life corals.

### Computational Approach

1. The model uses a 3D grid of size N × N × N, where N = 2 × rmax + 3
2. The seed to start the coral is placed at the center of the base of the grid (m, m, 0) 
3. Particles are spawned randomly on a sphere so that they're equidistant from the center. The spawning radius is slightly beyond the current growth region (rmin + 2) to provide a buffer for random movement.
4. Particles perform a random walk in 3D space until they stick to the cluster, or escape the growth region (in which case they're discarded)
5. the stickiness probability ('threshold') governs whether a particle attached to the coral. It is defined as: 

   threshold = k_cons × z + 0.2

  - z is the particle's height, in this way we bias the coral towards upward growth (mimicing the biological mechanism of growth towards light)
  - k_cons is a tunable parameter to adjust the likelihood of attachment
6. To determine if a particle is near the coral cluster, its adjacent cells are checked. If any neighbor is part of the cluster, the particle is allowed to stick.
7. The simulation stops when the cluster reaches a predefined maximum radius, or a predefined computational limit is reached 

#### Complexity
- space: the 3D grid requires O(N^3) memory 
- time: the random walk and neighbor check operations are the most computationally heavy 
- the use of Numba JIT compiler speeds up our operations 

#### Spherical coordinates

The spherical coordinates `(r, θ, ψ)` (radius, azimuthal angle, polar angle) are converted into Cartesian coordinates `(x, y, z)` for positioning the particles in the 3D grid. The mathematical relationship between spherical and Cartesian coordinates is given by:

- x = r × sin(ψ) × cos(θ)
- y = r × sin(ψ) × sin(θ)
- z = r × cos(ψ) 

Where:
- r: Radius (distance from the origin or center of the sphere).
- θ: Azimuthal angle (angle in the x-y plane, ranging from 0 to 2)
- θ: Azimuthal angle (angle in the x-y plane, ranging from 0 to 2×pi)
- ψ: Polar angle (angle from the z-axis, ranging from 0 to pi). 

#### 1.1 Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from numba import jit

#### 1.2 Basic 3D model

In [2]:
@jit(nopython=True, parallel=True)
def dla3D(rmin, k_cons):

    '''
    this function simulates a coral, grown using diffusion limited aggregation. 

    Inputs
    - rmin: the minimum growth radius that defines the region of growth; i.e. determines the cluster size
    - k_cons: controls the probability of particle attachment; determines density and connectedness within the coral 

    max_iter: maximum iterations. sets a limit of the simulation runtime.

    '''

    # for a sphere, volume is proportional to r^3, so rmax = rmin * (d ^ 1/3), where d scales the coral growth's volume
    d = 3
    max_iter = 50000
    rmax = round(rmin * (d ** (1 / 3)))

    # create a 3D grid and initialise it w 0s
    N = 2 * rmax + 3
    A = np.zeros((N, N, N), dtype=np.int32)

    # start the coral the the bottom-centre
    m = N // 2
    A[m, m, 0] = 1

    # mass tracks the n/o particals in the coral
    # ^ could be an evaluation criteria(?)
    mass = 1

    # flag to stop the simulation when needed
    terminate = False

    for _ in range(max_iter):

        # random starting position on a sphere around the cluster (so they're all equidistant from the seed)
        # new particle is spawned slightly beyond the growth boundary to give it some buffer zone to random-walk
        d = rmin + 2

        # random angle for x-y plane
        theta = np.random.uniform(0, 2 * np.pi)
        # random angle for z-axis
        psi = np.arccos(1 - 2 * random.random())
        # convert spherical to cartesian x, y, z (ensure z > or = 0, so above seabed)
        x = m + round(d * np.sin(psi) * np.cos(theta))
        y = m + round(d * np.sin(psi) * np.sin(theta))
        z = max(0, round(d * np.cos(psi)))

        # skip if particle starts already inside the cluster
        if A[x, y, z] == 1:
            continue

        while True:
            # sum the neighbors, if the sum is 0, it means the particle is not near the coral
            neighbors = (
                A[x + 1, y, z] + A[x - 1, y, z] + A[x, y + 1, z] +
                A[x, y - 1, z] + A[x, y, z + 1] + A[x, y, z - 1]
            )

            # determine threshold based on height z to make coral tend upwards
            threshold = k_cons * z + 0.2

            # stick the particle if it's near the cluster and satisfies stickiness probability
            if neighbors > 0 and random.random() < threshold:
                # stop the simulation if the cluster exceeds the limit
                if (x - m) ** 2 + (y - m) ** 2 + (z) ** 2 >= rmin ** 2:
                    terminate = True
                    
                # add the particle to the cluster
                A[x, y, z] = 1
                mass += 1

                # make sure the coral is one continuous entity (biologically accurate)
                for dx, dy, dz in [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]:
                    nx, ny, nz = x + dx, y + dy, z + dz
                    if 0 <= nx < N and 0 <= ny < N and 0 <= nz < N and A[nx, ny, nz] == 0:
                        A[nx, ny, nz] = 1

                # move on to the next particle
                break

            # remove the particle from its current position...
            A[x, y, z] = 0

            # ... and move it randomly to the new position...
            # LIGHT VARIABLE TO BE ADDED HERE TO BIAS THE DIRECTION OF GROWTH
            directions = np.array([(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)])
            idx = np.random.randint(0, len(directions))
            direction = directions[idx]
            nx, ny, nz = x + direction[0], y + direction[1], z + direction[2]
            nx, ny, nz = x + direction[0], y + direction[1], z + direction[2]

            # ... if it's uncoccupied
            if 0 <= nx < N and 0 <= ny < N and 0 <= nz < N and A[nx, ny, nz] == 0:
                x, y, z = nx, ny, nz
                A[x, y, z] = 1

            # remove the particle if it escapes the growth region
            if (x - m) ** 2 + (y - m) ** 2 + (z) ** 2 > rmax ** 2:
                A[x, y, z] = 0
                break

        # stop if the coral reaches the computational limit
        if terminate:
            break

    return mass, A

## 2. Measurement metrics

In [3]:
def mass(coral):
    """
    calculate the total number of particles (mass) in the coral.
    """
    return np.sum(coral)

def shape_features(coral):
    """
    calculate shape features of the coral: radius and asymmetry.
    
    - radius: maximum distance from the center to any particle, representing the coral's growth extent.
    - asymmetry: distance of the coral's centroid from the center of the grid, indicating uneven growth.
    
    output:
        radius: float, maximum distance from the center.
        asymmetry: float, centroid's deviation from the center.
    """
    N = coral.shape[0]  # grid size
    m = N // 2  # center of the grid

    # get coordinates of coral particles
    coordinates = np.argwhere(coral == 1)

    # radius: maximum distance from the center to any particle
    radius = np.max(np.sqrt((coordinates[:, 0] - m) ** 2 + 
                             (coordinates[:, 1] - m) ** 2 + 
                             (coordinates[:, 2]) ** 2))

    # centroid: average position of all particles
    centroid = np.mean(coordinates, axis=0)

    # asymmetry: distance of centroid from the center
    asymmetry = np.sqrt((centroid[0] - m) ** 2 + (centroid[1] - m) ** 2 + (centroid[2]) ** 2)

    return radius, asymmetry

def growth_rate(coral):
    """
    calculate the vertical growth rate by measuring the number of new particles added at each layer.
    
    output:
        growth_rate: list, number of new particles added at each z-layer.
    """
    z_layers = coral.shape[2]  # number of z-layers
    growth_rate = []

    for z in range(z_layers):
        particles_in_layer = np.sum(coral[:, :, z])
        growth_rate.append(particles_in_layer)

    return growth_rate

def surface_area_to_volume(coral):
    """
    calculate the surface area to volume ratio of the coral in order to test the complexity.
    a higher ratio suggests that the coral has a more branched structure.
    
    - surface area: number of exposed faces of particles.
    - volume: total number of particles in the coral.
    
    output:
        ratio: float, surface area to volume ratio.
    """
    neighbors = np.array([[1, 0, 0], [-1, 0, 0], [0, 1, 0],
                          [0, -1, 0], [0, 0, 1], [0, 0, -1]])
    surface_area = 0

    for x, y, z in np.argwhere(coral == 1):
        for dx, dy, dz in neighbors:
            nx, ny, nz = x + dx, y + dy, z + dz
            if 0 <= nx < coral.shape[0] and 0 <= ny < coral.shape[1] and 0 <= nz < coral.shape[2]:
                if coral[nx, ny, nz] == 0:
                    surface_area += 1

    volume = mass(coral)
    ratio = surface_area / volume if volume > 0 else 0

    return ratio

def fractal_dimension(coral):
    """
    calculate the fractal dimension using the box-counting method.
    
    - fractal dimension: measures the coral's fractal complexity or spatial filling.
    - method: divide the grid into boxes of decreasing size and count non-empty boxes.
    
    output:
        fractal dimension: float
    """
    N = coral.shape[0]  # grid size

    # list of box sizes to test
    box_sizes = [2 ** i for i in range(int(np.log2(N)) - 1, 0, -1)]
    box_counts = []

    for box_size in box_sizes:
        # number of boxes along each dimension
        num_boxes = N // box_size

        # count non-empty boxes
        count = 0
        for i in range(num_boxes):
            for j in range(num_boxes):
                for k in range(num_boxes):
                    # extract the sub-box
                    sub_box = coral[i * box_size:(i + 1) * box_size,
                                    j * box_size:(j + 1) * box_size,
                                    k * box_size:(k + 1) * box_size]
                    # check if sub-box contains any coral particles
                    if np.any(sub_box):
                        count += 1

        box_counts.append(count)

    # fit a line to log-log plot to calculate dimension
    log_box_sizes = np.log(1 / np.array(box_sizes))  # log of inverse box size
    log_box_counts = np.log(box_counts)  # log of box counts
    slope, _ = np.polyfit(log_box_sizes, log_box_counts, 1)  # slope is the fractal dimension

    return slope 


import numpy as np
from scipy.ndimage import label, distance_transform_edt

def calculate_metrics(coral):
    """
    Calculate various metrics for the given 3D coral structure.
    """
    # Total mass (number of occupied cells)
    mass = np.sum(coral)
    
    # Radius and asymmetry
    center = np.array(coral.shape) // 2
    occupied_points = np.array(np.nonzero(coral)).T  # All occupied points
    distances = np.linalg.norm(occupied_points - center, axis=1)  # Distances from the center
    radius = np.mean(distances)
    asymmetry = np.std(distances) / radius  # Normalized standard deviation
    
    # Fractal dimension
    box_sizes = [2, 4, 8, 16]
    box_counts = []
    for box_size in box_sizes:
        grid = coral[::box_size, ::box_size, ::box_size]
        box_counts.append(np.sum(grid > 0))
    fractal_dim = np.polyfit(np.log(box_sizes), np.log(box_counts), 1)[0]
    
    # Surface area to volume ratio
    surface_area = np.sum(np.logical_and(coral, ~distance_transform_edt(~coral, return_distances=False)))
    volume = mass
    sv_ratio = surface_area / volume if volume > 0 else 0
    
    # Number of branches and branch density
    labeled_coral, num_branches = label(coral)  # Label connected components
    branch_density = num_branches / volume if volume > 0 else 0
    
    # Overlap measure (if applicable, generally minimal for DLA)
    overlap = np.sum(coral > 1)
    
    # Average branch length
    branch_lengths = []
    for branch_label in range(1, num_branches + 1):
        branch = (labeled_coral == branch_label)
        branch_points = np.array(np.nonzero(branch)).T
        if len(branch_points) > 1:
            branch_length = np.linalg.norm(branch_points.max(axis=0) - branch_points.min(axis=0))
            branch_lengths.append(branch_length)
    avg_branch_length = np.mean(branch_lengths) if branch_lengths else 0
    
    # Growth anisotropy
    growth_along_axes = [
        np.sum(np.any(coral, axis=(1, 2))),  # Growth along Z-axis
        np.sum(np.any(coral, axis=(0, 2))),  # Growth along Y-axis
        np.sum(np.any(coral, axis=(0, 1)))   # Growth along X-axis
    ]
    anisotropy = max(growth_along_axes) / min(growth_along_axes) if min(growth_along_axes) > 0 else 0
    
    return {
        "mass": mass,
        "radius": radius,
        "asymmetry": asymmetry,
        "fractal_dim": fractal_dim,
        "sv_ratio": sv_ratio,
        "num_branches": num_branches,
        "branch_density": branch_density,
        "overlap": overlap,
        "avg_branch_length": avg_branch_length,
        "anisotropy": anisotropy
    }

## 3. Simulations

#### 3.1 The effect of light intensity

In [18]:
# Simulate DLA for three different k_cons values
mass1, A1 = dla3D(75, 0.05)
mass2, A2 = dla3D(75, 0.45)
mass3, A3 = dla3D(75, 0.85)

def extract_coordinates(A):
    X, Y, Z = [], [], []
    N = len(A)
    for i in range(N):
        for j in range(N):
            for l in range(N):
                if A[i, j, l] == 1:
                    X.append(i)
                    Y.append(j)
                    Z.append(l)
    return X, Y, Z

X1, Y1, Z1 = extract_coordinates(A1)
X2, Y2, Z2 = extract_coordinates(A2)
X3, Y3, Z3 = extract_coordinates(A3)

# Interactive backend for rotating the 3D grid
plt.switch_backend('QtAgg')

fig = plt.figure(figsize=(18, 6))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(X1, Y1, Z1, s=10, marker='o', c=Z1, cmap='viridis', alpha=0.7)
ax1.set_title("k = 0.05")
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Z")

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(X2, Y2, Z2, s=10, marker='o', c=Z2, cmap='viridis', alpha=0.7)
ax2.set_title("k = 0.45")
ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Z")

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(X3, Y3, Z3, s=10, marker='o', c=Z3, cmap='viridis', alpha=0.7)
ax3.set_title("k = 0.85")
ax3.set_xlabel("X")
ax3.set_ylabel("Y")
ax3.set_zlabel("Z")

plt.tight_layout()
plt.show()

x=37, y=65, z=9
x=-76, y=-6, z=3
x=74, y=15, z=2


In [19]:
coral_mass1 = mass(A1)
radius1, asymmetry1 = shape_features(A1)
fractal_dim1 = fractal_dimension(A1)
growth_rate_per_layer1 = growth_rate(A2)
sv_ratio1 = surface_area_to_volume(A1)

coral_mass2 = mass(A2)
radius2, asymmetry2 = shape_features(A2)
fractal_dim2 = fractal_dimension(A2)
growth_rate_per_layer2 = growth_rate(A2)
sv_ratio2 = surface_area_to_volume(A2)

coral_mass3 = mass(A3)
radius3, asymmetry3 = shape_features(A3)
fractal_dim3 = fractal_dimension(A3)
growth_rate_per_layer3 = growth_rate(A3)
sv_ratio3 = surface_area_to_volume(A3)

In [45]:
k_values = [0.05, 0.45, 0.85]

# Replace these with actual computed values
masses = [coral_mass1, coral_mass2, coral_mass3]
radii = [radius1, radius2, radius3]
asymmetries = [asymmetry1, asymmetry2, asymmetry3]
fractal_dimensions = [fractal_dim1, fractal_dim2, fractal_dim3]
sv_ratios = [sv_ratio1, sv_ratio2, sv_ratio3]

metrics = ['Mass', 'Radius', 'Asymmetry', 'Fractal Dimension', 'Surface Area to Volume Ratio']
values = [masses, radii, asymmetries, fractal_dimensions, sv_ratios]

fig, axs = plt.subplots(1, 5, figsize=(20, 5))

for i, ax in enumerate(axs):
    ax.scatter(k_values, values[i], color='blue', s=100)  # Scatter plot
    ax.set_title(metrics[i])
    ax.set_xlabel('k_cons')
    ax.set_ylabel(metrics[i])
    ax.set_xticks(k_values)

plt.tight_layout()
plt.show()